# MGnify tutorial
This tutorial is a self-guided journey through some of the feature of the [MGnify](https://www.ebi.ac.uk/metagenomics) platform.
We will look at how to search for, browse, and download different kinds of datasets derived from metagenomic samples.

This tutorial is a JupyterLite notebook, which means there is a Python interpreter running in your web browser. This requires an up to date browser. The Python that runs here is a bit different to "normal" Python, so keep that in mind if you're trying to write your own code here.

Most of the notebook is instructions and quiz questions, but there is also a bit of Python code if you're interested in how to access the data programatically. Knowing and using Python is not necessary to follow most of this tutorial.

<div style="background: #d0debb; padding: 16px; border-radius: 4px; margin: 8px">
To use this notebook, click the <b>Run</b> menu at the top, and then <b>Run all cells</b>.
</div>

You can also run individual cells using the Play icon at the top, or type shift-enter in any cell to run it.

For help, contact `sandyr@ebi.ac.uk`

---

# Part 1: Exploring MGnify Studies
## Task 1.1: Search for a MGnify study

Go to the [MGnify website](https://www.ebi.ac.uk/metagenomics), and **use the "Text Search" feature to search for studies of marine sediment collected using an "ROV" (that's a _remote operated vehicle_) from the Environmental > Aquatic > Marine > Sediment biome.**

_Hint: the use of an ROV is only mentioned in the study description, there is no specific metadata field for this._

In [ ]:
%pip install -q ipywidgets

In [ ]:
from utils.quiz import check_answer, _encode_answer, question

In [ ]:
question(
    "What is the MGnify Study accession (MGYS) of the first study that matches those parameters?",
    "MGYSxxx",
    [b'bWd5czAwMDA1ODUx']
)

## Task 1.2: Find metadata for a study

Follow the link on the study result above, to reach the detail page for that study on MGnify.



In [ ]:
question(
    "Off the coast of which continent were most the study's samples collected?",
    "",
    [b'c291dGggYW1lcmljYQ=='],
    ["Africa", "Antarctica", "Asia", "Europe", "North America", "Oceania", "South America"]
)

Not all metadata are neatly organised in structured fields!

Very often, the information you're interested in is only mentioned in the text descriptions of studies and samples uploaded to ENA, or even only in the "Methods" section of a publication.

**Use the "metadata from Europe PMC Annotations" feature to discover published mentions of metadata about this study.**

In [ ]:
question(
    "What is the deepest sampling *site* mentioned in the publication annotations?",
    "",
    [b'OTMwMCAtIDEwMDEwIG0='],
    ["1000 m", "2560 m", "7720 - 8085 m", "9300 - 10010 m"]
)

## Task 1.3: Finding common taxa in a study

MGnify produces "Analysis summaries" for each Study. These summarise the taxonomic annotations over all of the sequencing runs (or assemblies) in the study. They are useful for getting a quick overview of the count of taxa present in each analysed sample.

For a study of amplicon sequences, sometimes there will be multiple analysis summaries originating from different databases (e.g. SSU, LSU, ITSone).

**For the study [MGYS00005851](https://www.ebi.ac.uk/metagenomics/studies/MGYS00005851#analysis), find the most common Bacterial phyla using the SSU taxonomic summary.**

You could do this by downloading a TSV file and analysing it using something like Excel or R.

Or, you can use the following Python code for a programmatic approach.

In [ ]:
import json
from js import fetch
import pandas as pd
from io import StringIO

In [ ]:
# Download the TSV file programmatically:
res = await fetch('https://www.ebi.ac.uk/metagenomics/api/v1/studies/MGYS00005851/pipelines/5.0/file/ERP116704_phylum_taxonomy_abundances_SSU_v5.0.tsv')
text = await res.text()
# in a "normal" Python environment, this could be e.g. res = requests.get("https://..."); text = res.text()
# here we use js.fetch since we're running Python in the browser.

# Read the TSV into a pandas dataframe
itsone_taxonomies = pd.read_csv(StringIO(text), sep='\t')
itsone_taxonomies

In [ ]:
# Sum the taxonomy counts across all runs in the study
itsone_taxonomies['study_total'] = itsone_taxonomies.filter(like="ERR").sum(axis=1)

# Limit to phyla in the bacteria superkingdom
itsone_bacteria = itsone_taxonomies[itsone_taxonomies.superkingdom == 'Bacteria']
itsone_bacteria.set_index('phylum', inplace=True)

# Show just the most prevalent phyla
itsone_bacteria.sort_values(by="study_total", ascending=False).head()

## Task 1.4: Think about the caveats...

**Was what we just did (summing bacterial tax counts across all samples of the study) a good idea?**

Things to consider: (Hint: Check the study page's "Description" and publication annotations from Schauberger _et al._)
1. Were all of the sequencing runs targetting the 16s SSU region, so designed to barcode bacteria?
2. Were the primers used likely to sequence the same taxa across all samples?

<details>
<summary>Tell me</summary>
The study description and publication mention that all samples were prepared with primers targetting SSUs, but that some were targetting 16s and some 18s. This means that it <b>is</b> fair to be only using the SSU taxonomic analysis summary across the study. However, it also mentions that some samples were prepared with primers specific to archaea, so there is likely a bias to different taxa in different samples. (In fact, a few different primers were used as can be seen by digging further into the metadata of the study, samples and publication.) This means our basic summation across all samples is likely not an unbiased measurement.
</details>

---

# Part 2: Exploring MGnify Analyses

## Task 2.1: Search for a MGnify analysis

Let's say we're particularly interested in Chloroflexi phylum (also known as Chloroflexota) – the third most prevalent Bacterial phylum in the study we were looking at in Part 1. 

Go to the [MGnify website](https://www.ebi.ac.uk/metagenomics/search/analyses), and **use the "Text Search" feature to search for _Sample analyses_, that contain Organisms in the Bacteria > Chloroflexi lineage, also collected from the Environmental > Aquatic > Marine > Sediment biome. Limit the search to samples analysed with MGnify's pipeline version 5.0 (the latest).**

In [ ]:
question(
    "Look at the 'Experiment type' filter on the left of the search results: there are some <b>assembly</b> analyses.<br/>Why might we want to look into these next?",
    "",
    [b'dG8gZGlzY292ZXIgZnVuY3Rpb25hbCBhbm5vdGF0aW9ucw=='],
    ["To browse complete genomes", "To get more taxonomic diversity", "To discover functional annotations"]
)

## Task 2.2: Find annotations within an analysis
**Limit your search to just those Assembly analyses.**

There are more filters we could use here, like location, or filtering to find analysis where a specific GO or InterPro functional annotation has been found on the assembled contigs.

For now, **scroll through the list to find MGYA00594115, and click it to open the analysis page.**

In [ ]:
question(
    "According to the charts in the <b>Taxonomic analysis</b> section's SSU results, what fraction of the phylum composition is assigned to Chloroflexi?",
    "x.yy %",
    [b'NC40OSAl', b'NC40OSU=', b'NC40OQ=='],
)

Now turn to the functional analysis information, in particular **the Pathways/Systems section.** Functional annotations of a metagenomic assembly are based on comparison of proteins coding sequences found in the assembly's contigs to databases of protein function such as KEGG orthologs. KEGG modules are pathways of KEGG orthologs, and a pathway's completeness is determined by the presence or absence of the minimum number of orthologs present to complete that pathway.

In [ ]:
question(
    "What is the first 'complete' KEGG Module for this assembly analysis?<br/>(Complete means all of the Kegg Orthologs in that module are present in the assembly.)",
    "Mxxxxx",
    [b'bTAwMDAx', b'MDAwMDE='],
)

## Task 2.3: Think about what this means...

**Does this mean that the Kegg Module you just found is necesarily a function performed by organisms within the taxa we originally searched for (Chloroflexi)?**

_Consider that the functional annotations are on contigs from the entire assembly..._

---

# Part 3: Metagenomic Assembled Genomes

To find functional annotations for a specific species, we can use metagenome assembly genomes (MAGs) – draft genomes made by taxonomically binning assembled contigs.
[MGnify Genomes](https://www.ebi.ac.uk/metagenomics/browse/genomes) is MGnify's resource for MAGs, where MAGs derived from various biomes are organised into species-level clusters and annotated with a suite of functional annotation tools.

## Task 3.1: Find Chloroflexi MAGs

**Go to the [MGnify Genomes website](https://www.ebi.ac.uk/metagenomics/browse/genomes)**.

Given the biome we started this journey from, **which of the MGnify Genomes catalogues would you guess we're mostly like to find MAGs from the Chloroflexi phylum in?**

Let's check. **Go to the "All genomes" list, and use the Filter to search for `Chloroflex`** (since this will catch lineages using either the `Chloroflexi` or `Chloroflexota` naming scheme).

In [ ]:
question(
    "Which catalogue are most of the MAGs found in?",
    "",
    [b'bWFyaW5l'],
    ["Marine", "Human Oral", "Zebrafish fecal"]
)

## Task 3.2: Check functional annotations of a MAG

From the list, **open the MAG for the species `Casp-Chloro-G4`**.

In [ ]:
question(
    "What is the most prevalent KEGG Module for this genome?",
    "Mxxxxx",
    [b'bTAwMTc4', b'MDAxNzg='],
)

_Note that there are "Download" files available for both the assembly analysis we looked at before, and for this genome analysis, that include full listings of the Kegg annotations amongst others. You could download these files similarly to the study summary TSV file we analysed in Part 1, to compare these programmatically._

## Task 3.3: Cross-catalogue searching
Looking at all the data you have available for this MAG, and exploring the other features available in the MGnify Genomes resource, can you work out how you might search for genomes similar to this one in other catalogues?

<details>
<summary>Tell me</summary>
The <a href="https://www.ebi.ac.uk/metagenomics/browse/genomes?browse-by=mag-search">"MAG search" feature</a> uses <a href="https://sourmash.readthedocs.io/en/latest/">Sourmash</a> to search genomes against genomes, based on sequence similarity. You could download the FASTA file of the MAG MGYG000297014, from its Downloads tab, and then upload this as the query genome to the MAG Search, selecting all catalogues as the target. At the time of writing, this search will only match the genome we downloaded from the marine catalogue.
</details>

---

# Further reading:

### Latest MGnify publication

> Richardson L, Allen B, Baldi G, et al. [MGnify: the microbiome sequence data analysis resource in 2023.](https://europepmc.org/article/PMC/PMC9825492) Nucleic Acids Research. 2023 Jan;51(D1):D753-D759. DOI: 10.1093/nar/gkac1080. PMID: 36477304; PMCID: PMC9825492.

### MGnify Genomes publication
> Gurbich TA, Almeida A, Beracochea M, et al. [MGnify Genomes: A Resource for Biome-specific Microbial Genome Catalogues.](https://europepmc.org/article/MED/36806692) Journal of Molecular Biology. 2023 Jul;435(14):168016. DOI: 10.1016/j.jmb.2023.168016. PMID: 36806692; PMCID: PMC10318097.


### Documentation
> [docs.mgnify.org](https://docs.mgnify.org)

### MGnify training courses
> [MGnify on EBI Training](https://www.ebi.ac.uk/training/search-results?query=mgnify)

### Updates
> [@MGnifyDB](https://twitter.com/mgnifyDB)